In [3]:
import numpy as np
import dask
import dask.dataframe as dd
from dask.distributed import Client
import joblib
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.svm import LinearSVC

In [4]:
client = Client('172.31.88.97:8786')
client = client.restart()
print(client)

<Client: 'tcp://172.31.88.97:8786' processes=0 threads=0, memory=0 B>


In [7]:
data = dd.read_csv('/home/ubuntu/data/cleaned_data.csv', sample=2**30, assume_missing=False)

In [8]:
data.size.compute()

FileNotFoundError: [Errno 2] No such file or directory: '/home/ubuntu/data/cleaned_data.csv'

In [9]:
data = data.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1)

In [ ]:
data.head()

In [7]:
data['dateDelta'] = data['flightDate'] - data['searchDate']

In [8]:
# Split data into X and y
X_reg = np.asarray(data[['dateDelta', 'seatsRemaining', 'totalTravelDistance',
                  'durationSeconds']])

y_reg = np.asarray(data['baseFare'])

X_class = np.asarray(data[['dateDelta', 'seatsRemaining', 'totalTravelDistance',
                  'durationSeconds', 'baseFare']])

y_class = np.asarray(data['destinationAirport'])

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.2, random_state=5110)
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X_class, y_class, test_size=0.2, random_state=5110)

In [18]:
#KNN model
with joblib.parallel_backend('dask'):
    knn_model = KNeighborsRegressor(n_neighbors=5)
    knn_model.fit(X_train_reg, y_train_reg)
    acc = knn_model.score(X_test_reg, y_test_reg)

In [10]:
acc

0.7355402301680947

In [19]:
#K-Means
with joblib.parallel_backend('dask'):
    kmeans_model = KMeans(n_clusters=2, random_state=5110, n_init="auto")
    kmeans_model.fit(X_train_reg)
   

In [12]:
#Random Forest
with joblib.parallel_backend('dask'):
    rf_model = RandomForestRegressor(max_depth=2, random_state=5110)
    rf_model.fit(X_train_reg, y_train_reg)
   
    y_pred = rf_model.predict(X_test_reg)
    mse = mean_squared_error(y_test_reg, y_pred)
   

In [14]:
np.sqrt(mse)

184.2734522424671

In [16]:
data.baseFare.describe()

count    500000.000000
mean        322.205953
std         214.183514
min          18.020000
25%         180.470000
50%         292.090000
75%         413.020000
max        2968.370000
Name: baseFare, dtype: float64

In [ ]:
#SVM
with joblib.parallel_backend('dask'):
    svm_model = LinearSVC(random_state=5110, multi_class='ovr')
    svm_model.fit(X_train_class, y_train_class)
    acc = svm_model.score(X_test_class, y_test_class)

/home/ubuntu/.local/lib/python3.10/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
